In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

#import lightgbm as lgb
import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

# Preprocessing

In [5]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [6]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [7]:
pseudo = pd.read_csv("../input/in-python-tabular-denoising-residual-network/submission_v2.csv")[target]
test_pseudo = pd.concat([test, pseudo], axis=1)
all_pseudo = pd.concat([train, test_pseudo]).reset_index(drop=True)

# Optuna

In [8]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [9]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo[scale_features], y=all_pseudo[target]):
  """
  """
  param_space = {
                'objective': 'reg:squarederror',              
                  'learning_rate':trial.suggest_uniform('learning_rate', 1e-3, 1e-1),        
                  'n_estimators':N_ESTIMATORS,       
                  'subsample': trial.suggest_uniform('subsample', 0.2, 1.0),      
                  'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.2, 1.0),   
                  'max_depth':trial.suggest_int('max_depth', 2, 30),         
                  'lambda':trial.suggest_uniform('lambda', 0.01, 50),     
                  'alpha':trial.suggest_uniform('alpha', 0.01, 50),    
                  'min_child_weight':trial.suggest_uniform('min_child_weight', 100, 500),
                  'importance_type': 'total_gain',      
                  'tree_method': 'hist'    
     
                }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  xgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo[scale_features].iloc[trn_idx], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_rn(X_train)

  X_valid, y_valid = all_pseudo[scale_features].iloc[oof_idx], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]

  #start = time.time()
  for inseed in seed_list:
    param_space['seed'] = inseed

    model = xgb.XGBRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )


    xgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, xgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [10]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-26 03:28:30,162] A new study created in memory with name: no-name-8e7a6280-1c27-4652-9f49-bce5bfe71f2d


[0]	validation_0-rmse:9.74579
[270]	validation_0-rmse:7.81381
[0]	validation_0-rmse:9.74561
[282]	validation_0-rmse:7.83046


[I 2021-08-26 03:31:12,620] Trial 0 finished with value: 7.778687523691052 and parameters: {'learning_rate': 0.07888169981385038, 'subsample': 0.48682573932049755, 'colsample_bytree': 0.9244274130142915, 'max_depth': 28, 'lambda': 0.8642522171349019, 'alpha': 22.587159507652558, 'min_child_weight': 356.6883537487601}. Best is trial 0 with value: 7.778687523691052.


[0]	validation_0-rmse:9.80291
[322]	validation_0-rmse:7.81810
[0]	validation_0-rmse:9.80374
[310]	validation_0-rmse:7.82517


[I 2021-08-26 03:34:41,213] Trial 1 finished with value: 7.783485130807614 and parameters: {'learning_rate': 0.06386189933050504, 'subsample': 0.4917525605777396, 'colsample_bytree': 0.33898944235427886, 'max_depth': 29, 'lambda': 14.804888865510152, 'alpha': 37.89542748792464, 'min_child_weight': 136.28867727681455}. Best is trial 0 with value: 7.778687523691052.


...

[0]	validation_0-rmse:9.83182
[710]	validation_0-rmse:7.76337
[0]	validation_0-rmse:9.83159
[685]	validation_0-rmse:7.76435


[I 2021-08-26 10:32:28,412] Trial 28 finished with value: 7.757350792426479 and parameters: {'learning_rate': 0.05596440407601175, 'subsample': 0.9182082639671065, 'colsample_bytree': 0.7993260340378379, 'max_depth': 8, 'lambda': 1.8223313995973616, 'alpha': 11.875875593666029, 'min_child_weight': 337.88483908494027}. Best is trial 23 with value: 7.748372011001994.


[0]	validation_0-rmse:9.98201
[1000]	validation_0-rmse:7.80444
[2000]	validation_0-rmse:7.78671
[3000]	validation_0-rmse:7.77850
[4000]	validation_0-rmse:7.77359
[5000]	validation_0-rmse:7.77008
[6000]	validation_0-rmse:7.76802
[7000]	validation_0-rmse:7.76629
[8000]	validation_0-rmse:7.76495
[9000]	validation_0-rmse:7.76388
[9999]	validation_0-rmse:7.76311
[0]	validation_0-rmse:9.98203
[1000]	validation_0-rmse:7.80450
[2000]	validation_0-rmse:7.78714
[3000]	validation_0-rmse:7.77880
[4000]	validation_0-rmse:7.77380
[5000]	validation_0-rmse:7.77098
[6000]	validation_0-rmse:7.76872
[7000]	validation_0-rmse:7.76688
[8000]	validation_0-rmse:7.76559
[9000]	validation_0-rmse:7.76448
[9999]	validation_0-rmse:7.76369


[I 2021-08-26 10:54:55,672] Trial 29 finished with value: 7.763246705425261 and parameters: {'learning_rate': 0.017213131792756825, 'subsample': 0.8538757746277529, 'colsample_bytree': 0.9494973870076724, 'max_depth': 2, 'lambda': 0.6430543633550476, 'alpha': 18.265843195660864, 'min_child_weight': 395.1536177644449}. Best is trial 23 with value: 7.748372011001994.


Number of finished trials: 30
Best trial: {'learning_rate': 0.009137980241462154, 'subsample': 0.894577526126585, 'colsample_bytree': 0.631933960621113, 'max_depth': 14, 'lambda': 12.655747040153953, 'alpha': 8.692284168970499, 'min_child_weight': 256.30230889039706}


In [11]:
study.best_params

{'learning_rate': 0.009137980241462154,
 'subsample': 0.894577526126585,
 'colsample_bytree': 0.631933960621113,
 'max_depth': 14,
 'lambda': 12.655747040153953,
 'alpha': 8.692284168970499,
 'min_child_weight': 256.30230889039706}

# Log

7.758238638554282 row-wise noise ver1

7.758171791679289 random noise ver4

7.745837750406564 no noise ver5

7.744947212695728 no noise ver7 (narrow space)

7.744636762159451 no noise ver8 (narrow space) final

=============================================
============== Change pseudo ================

